In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



import tensorflow as tf



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Datasets Loading.....

In [ ]:
df = pd.read_csv('/kaggle/input/fake-news/train.csv')

In [ ]:
test_df = pd.read_csv('/kaggle/input/fake-news/test.csv')

In [ ]:


main_df = pd.concat([df.drop(['label'],axis=1),test_df],axis=0)

In [ ]:
main_df.shape

## Missing Data

In [ ]:
main_df.title.head(5)

In [ ]:
main_df.text.head(5)

In [ ]:
main_df.title=main_df.title.fillna(main_df['text'])

In [ ]:
main_df.isnull().sum()

In [ ]:
main_df.text=main_df.text.fillna(main_df.title)

In [ ]:
main_df.isnull().sum()

In [ ]:
main_df[main_df.author.isnull()]

In [ ]:
main_df.author = main_df.author.fillna('unknown')

In [ ]:
main_df.isnull().sum()

In [ ]:
X= main_df
y = df['label']

## Pre_processing...

In [ ]:
from tensorflow.keras.layers import Embedding ,LSTM , Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [ ]:
voc_size= 5000

In [ ]:
messages = X.copy()

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


In [ ]:
nltk.download('stopwords')

In [ ]:
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review =review.lower()
    review = review.split()
    review = [ps.stem(word)for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [ ]:
one_represent = [one_hot(words,voc_size) for words in corpus]

In [ ]:
one_represent

In [ ]:
sent_length = 20
embedded_docs = pad_sequences(one_represent,padding='pre',maxlen=sent_length)
print(embedded_docs)

## Model creating...

In [ ]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
len(embedded_docs),y.shape


In [ ]:
X_final=np.array(embedded_docs)
X_train_final  = X_final[:20800]

X_test_final   = X_final[20800:]
y_final = np.array(y)
X_test_final.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test =train_test_split(X_train_final ,y_final ,test_size=0.33,random_state=42)


In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

## predictions...

In [ ]:
y_pred = model.predict_classes(X_test_final)


In [ ]:

y_pred.shape,test_df.id.shape

In [ ]:
submission.head()

In [ ]:
y_pred=y_pred.reshape(5200)
type(y_pred)


In [ ]:
submission = pd.DataFrame({'id':test_df.id,'label':y_pred})

In [ ]:
submission.head()

In [ ]:
submission.shape

In [ ]:
submission.to_csv('LSMT_model_work_0.90.csv',index=False)